# Problema 6

In [1]:
# Importando bibliotecas
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Configura o cliente Open-Meteo API com cache e tenta novamente em caso de erro
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Certifica de que todas as variáveis ​​meteorológicas necessárias estejam listadas 
# A ordem das variáveis ​​em horária ou diária é importante para atribuí-las corretamente abaixo
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": -22.9064,
	"longitude": -43.1822,
	"hourly": "pressure_msl",
	"timezone": "America/Sao_Paulo"
}
responses = openmeteo.weather_api(url, params=params)

# Processa o primeiro local. Adiciona um for-loop para vários locais ou modelos climáticos
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Processa os dados. A ordem das variáveis ​​precisa ser a mesma solicitada.
hourly = response.Hourly()
hourly_pressure_msl = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["pressure_msl"] = hourly_pressure_msl

# Cria um DataFrame
previsao_pressao_atm = pd.DataFrame(data = hourly_data)
previsao_pressao_atm.head()

Coordinates -22.75°N -43.0°E
Elevation 12.0 m asl
Timezone b'America/Sao_Paulo' b'-03'
Timezone difference to GMT+0 -10800 s


,date,pressure_msl
0,2024-10-21 03:00:00+00:00,1016.799988
1,2024-10-21 04:00:00+00:00,1016.099976
2,2024-10-21 05:00:00+00:00,1015.799988
3,2024-10-21 06:00:00+00:00,1016.000000
4,2024-10-21 07:00:00+00:00,1016.099976


In [2]:
# Altera o nome das colunas
previsao_pressao_atm.columns = ['momento', 'valor']

In [3]:
# Conecta com o banco de dados
import mysql.connector

conexao = mysql.connector.connect(host='localhost',user='root',password='9090#',database='pressao_atm' )
cursor = conexao.cursor()


In [4]:
# Cria a tabela no banco de dados
query_create_table = """
CREATE TABLE IF NOT EXISTS pressao_atm.previsao_pressao_atm (
    id INT NOT NULL AUTO_INCREMENT,
    momento TIMESTAMP,
    valor FLOAT,
    PRIMARY KEY(id)
);
"""
cursor.execute(query_create_table)



In [5]:
# Insere os dados do DataFrame na tabela do MySQL
for _, row in previsao_pressao_atm.iterrows():
    momento = row['momento']
    valor = row['valor']
    
    query_insert = """
    INSERT INTO pressao_atm.previsao_pressao_atm (momento, valor)
    VALUES (%s, %s);
    """
    
    cursor.execute(query_insert, (momento, valor))

# Confirma a inserção 
conexao.commit()
cursor.close()
conexao.close()
